<a href="https://colab.research.google.com/github/pedro-delima/Protected-World/blob/main/protected_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -q google-adk
!pip install pyowm

In [13]:

import os
from google.colab import userdata

os.environ['OWM_API_KEY'] = userdata.get('OWM_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
openweathermap_key = '5b78a5f8fcb842356ad4c355bd4daa99'


In [15]:

from geopy.geocoders import Nominatim
from google import genai
from pyowm import OWM
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings


In [16]:

def menu():
    print('-------------------------')
    print('PROTECTED WORLD')
    print('-------------------------')
    print('Saiba antes, aja a tempo. Protected World, seu aliado em uma vida protegida e confortável')
    global local
    local = input('Informe o nome da cidade em que você está: \n')
    coordenadas(local)


In [17]:

def coordenadas(endereco):
    geolocator = Nominatim(user_agent="protected_world")
    localizacao = geolocator.geocode(endereco)
    informacoes_climaticas(localizacao.latitude, localizacao.longitude)


In [18]:

def informacoes_climaticas(latitude, longitude):
    owm = OWM(openweathermap_key)
    mgr = owm.weather_manager()
    observation_coords = mgr.weather_at_coords(latitude, longitude)
    w = observation_coords.weather
    global temperatura
    temperatura = w.temperature('celsius')['temp']
    sensacao_termica = w.temperature('celsius')['feels_like']
    global umidade
    umidade = w.humidity
    global velocidade_vento
    velocidade_vento = w.wind()['speed']
    direcao_vento = w.wind()['deg']
    status_geral = w.detailed_status
    chuva = w.rain if w.rain else {}
    horario_nascer_sol = w.sunrise_time('iso')
    horario_por_do_sol = w.sunset_time('iso')
    pressao = w.pressure['press']
    nuvens = w.clouds
    visibilidade = w.visibility_distance

    global dados_climaticos_formatados
    dados_climaticos_formatados = f"""
    Temperatura: {temperatura:.2f}°C (Sensação térmica: {sensacao_termica:.2f}°C)
    Umidade: {umidade}%
    Vento: {velocidade_vento:.2f} m/s
    Condição geral: {status_geral.capitalize()}
    Pressão: {pressao} hPa
    Cobertura de nuvens: {nuvens}%
    Chuva: {chuva}
    Visibilidade: {visibilidade} metros
    Nascer do sol: {horario_nascer_sol}
    Pôr do sol: {horario_por_do_sol}
    """
    analisar_clima_com_gemini(local, dados_climaticos_formatados)


In [19]:

def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response


In [20]:

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [26]:

# Função para analisar clima com agente Gemini
def analisar_clima_com_gemini(local, dados_climaticos_formatados):
    agente_analizador = Agent(
        name='Agente_analizador',
        model='gemini-2.0-flash',
        description='Agente analizador das condicoes climaticas',
        instruction="""objetivo: Analisar os dados climáticos fornecidos para a localização atual (Fortaleza, Ceará, Brasil) e determinar o risco de desastres climáticos iminentes, além de fornecer recomendações práticas para que o usuário possa manter a melhor qualidade de vida possível, dadas as condições meteorológicas presentes.

Dados a serem analisados:

O agente receberá um conjunto de dados climáticos atuais, incluindo (mas não se limitando a):

Temperatura: (em graus Celsius e sensação térmica)
Umidade: (em porcentagem)
Velocidade e Direção do Vento: (em m/s e graus/direção)
Condição Geral do Tempo: (descrição textual: ex: céu limpo, chuva leve, tempestade)
Pressão Atmosférica: (em hPa)
Cobertura de Nuvens: (em porcentagem)
Visibilidade: (em metros)
Precipitação: (se disponível, quantidade e probabilidade)
Qualidade do Ar: (se disponível, índice e principais poluentes)
Tarefas do Agente:

Avaliação de Risco de Desastre Climático:

Analisar os dados atuais em busca de indicadores de eventos climáticos perigosos. Considere os seguintes limiares (estes podem ser ajustados com base no conhecimento climático específico de Fortaleza):
Temperatura Extrema: Acima de 35°C (risco de insolação e desidratação).
Ventos Fortes: Acima de 15 m/s (risco de quedas de objetos, dificuldade de locomoção).
Chuva Intensa: Precipitação acima de 10 mm/h ou previsão de grande volume acumulado em curto período (risco de alagamentos).
Tempestades: Presença de raios, trovões e ventos fortes (risco de descargas elétricas, quedas de árvores).
Qualidade do Ar: Índice de Qualidade do Ar (ICA) em níveis considerados insalubres ou perigosos (risco para a saúde respiratória).
Considerar a combinação de fatores. Por exemplo, alta temperatura combinada com alta umidade aumenta o risco de problemas de saúde relacionados ao calor.
Emitir um alerta claro e conciso caso seja identificado um risco significativo de desastre climático iminente. O alerta deve especificar a natureza do risco (ex: "Alerta de calor extremo", "Risco de ventos fortes e tempestades").
Recomendações para Melhor Qualidade de Vida:

Com base nas condições climáticas atuais, fornecer recomendações práticas e acionáveis para o usuário. As recomendações devem abordar os seguintes aspectos:
Saúde e Bem-Estar:
Em caso de calor: Sugerir hidratação constante, evitar exposição prolongada ao sol, buscar locais frescos e ventilados, usar roupas leves e protetor solar.
Em caso de frio (menos comum em Fortaleza, mas possível em certas épocas): Recomendar agasalhar-se adequadamente.
Em caso de alta umidade: Sugerir manter ambientes ventilados para evitar a proliferação de mofo e ácaros.
Em caso de má qualidade do ar: Recomendar evitar atividades ao ar livre, usar máscaras de proteção (se apropriado), manter janelas fechadas.
Segurança:
Em caso de ventos fortes: Aconselhar evitar áreas com risco de queda de objetos (árvores, postes), fechar janelas e portas.
Em caso de chuva forte ou tempestade: Recomendar evitar áreas alagadas, buscar abrigo seguro, evitar contato com água e objetos metálicos.
Conforto:
Sugerir o uso de roupas e calçados adequados à temperatura e umidade.
Recomendar o uso de ventiladores ou ar condicionado, se necessário.
Atividades:
Sugerir horários mais adequados para atividades ao ar livre (evitando horários de pico de calor ou chuva).
Incentivar atividades em locais protegidos em condições climáticas adversas.
Formato da Resposta:

A resposta do agente deve ser clara, concisa e organizada, contendo as seguintes seções:

Avaliação de Risco: (Se houver risco, indicar claramente qual e o nível de severidade, se possível).
Recomendações para Qualidade de Vida: (Listar as recomendações de forma clara e com marcadores para fácil leitura).
Exemplo de Resposta:

"Avaliação de Risco: Alerta de calor extremo. A temperatura atual é de 36°C com sensação térmica elevada, representando um risco significativo de insolação e desidratação.

Recomendações para Qualidade de Vida:

Beba bastante água ao longo do dia para manter-se hidratado.
Evite a exposição prolongada ao sol, especialmente entre as 10h e as 16h.
Procure locais frescos e com boa ventilação.
Use roupas leves, claras e folgadas.
Aplique protetor solar com fator adequado.
Evite atividades físicas intensas durante os horários mais quentes."
Considerações Adicionais:

O agente deve ser capaz de adaptar as recomendações com base na combinação específica de condições climáticas presentes.
A linguagem utilizada deve ser acessível e fácil de entender para o usuário.
Se possível, fornecer informações adicionais relevantes, como a previsão para as próximas horas, para auxiliar o usuário no planejamento.
        """
    )
    entrada_agente_analizador = f'Local: {local}\n Dados Climaticos: {dados_climaticos_formatados}'
    analizador = call_agent(agente_analizador, entrada_agente_analizador)
    return analizador


In [29]:

def sugestoes_atividades(local, analizador):
    agente_atividades = Agent(
        name='Agente_Atividades',
        model='gemini-2.0-flash',
        description='Agente de buscar noticias no google',
        tools=[google_search],
        instruction="""Você é um especialista em atividades urbanas em determinada cidade fornecida pelo usuario. Sua função é dupla:

Recomendar uma atividade específica adequada às condições climáticas atuais ou previstas nessa cidade.
Pesquisar e fornecer os nomes específicos de locais (estabelecimentos comerciais ou espaços públicos) na cidade onde o usuário pode realizar essa atividade.
Ao recomendar a atividade, considere os horários mais adequados para atividades ao ar livre, evitando o pico de calor ou chuva, e sugira alternativas em locais protegidos para condições climáticas adversas.

Formato da Resposta:

Atividade Recomendada: [Nome da atividade]

Locais para Realizar a Atividade:

[Nome do Local 1]
[Nome do Local 2]
[Nome do Local 3] ...
Exemplo de interação:

Usuário (implícito): (Considerando um dia de sol por volta das 9h da manhã em Fortaleza)

Agente de IA (resposta):

Atividade Recomendada: Caminhada ou corrida leve na Beira-Mar.

Locais para Realizar a Atividade:

Calçadão da Praia de Iracema
Calçadão da Praia do Meireles
Calçadão da Praia de Mucuripe
Usuário (implícito): (Considerando uma tarde chuvosa em Fortaleza)

Agente de IA (resposta):

Atividade Recomendada: Visitar o Centro Dragão do Mar de Arte e Cultura.

Locais para Realizar a Atividade:
separe os locais por virgulo

Museu de Arte Contemporânea do Ceará
Planetário Rubens de Azevedo
Cinema do Dragão"""
    )
    entrada_do_agente_atividade = f"Local:{local}\n Recomendacoes IA de acordo com o clima: {analizador}"
    sugestao_atividade = call_agent(agente_atividades, entrada_do_agente_atividade)
    return sugestao_atividade


In [30]:

# Execução
menu()

print('################################')
print('Resultado da situação climatica')
print('################################')
if temperatura > 30:
    print(f'Temperatura: {temperatura:.2f}°C 🥵')
elif temperatura > 20:
    print(f'Temperatura: {temperatura:.2f}°C 😎')
elif temperatura > 10:
    print(f'Temperatura: {temperatura:.2f}°C 🥶')
else:
    print(f'Temperatura: {temperatura:.2f}°C ☃️')

if umidade > 50:
    print(f'Umiade: {umidade}% 💦')
else:
    print(f'Umidade: {umidade}% 🏜️')

analizador = analisar_clima_com_gemini(local, dados_climaticos_formatados)
display(to_markdown(analizador))
print('--------------------------------------------------------------------')

recomendacaoAtividade = input('Deseja receber indicações de atividades para fazer no dia de hoje? (Digite sim ou não)')
if recomendacaoAtividade.upper() == 'SIM':
    print('################################')
    print('Sugestão de Atividades')
    print('################################')
    sugestao_atividade = sugestoes_atividades(local, analizador)
    display(to_markdown(sugestao_atividade))
    print('--------------------------------------------------------------------')


-------------------------
PROTECTED WORLD
-------------------------
Saiba antes, aja a tempo. Protected World, seu aliado em uma vida protegida e confortável
Informe o nome da cidade em que você está: 
São Paulo
################################
Resultado da situação climatica
################################
Temperatura: 19.51°C 🥶
Umiade: 87% 💦


> **Avaliação de Risco:**
> 
> Não há risco imediato de desastre climático. As condições climáticas indicam um dia nublado e úmido, com temperatura amena e ventos fracos.
> 
> **Recomendações para Qualidade de Vida:**
> 
> *   **Conforto:**
>     *   Vista-se adequadamente para uma temperatura de 19.5°C. Uma jaqueta leve ou um casaco podem ser necessários, especialmente para quem sente mais frio.
>     *   Considere usar roupas que sequem rápido devido à alta umidade.
> *   **Saúde e Bem-Estar:**
>     *   Mantenha os ambientes ventilados para reduzir a sensação de umidade e prevenir a proliferação de mofo.
>     *   Apesar da ausência de chuva, tenha um guarda-chuva à mão, pois o tempo pode mudar rapidamente.
> *   **Atividades:**
>     *   Aproveite o dia para atividades internas ou ao ar livre que não exigem exposição direta ao sol, como visitas a museus, shoppings ou caminhadas em parques com áreas cobertas.
>     *   Verifique a previsão do tempo atualizada antes de planejar atividades ao ar livre mais longas.


--------------------------------------------------------------------
Deseja receber indicações de atividades para fazer no dia de hoje? (Digite sim ou não)sim
################################
Sugestão de Atividades
################################


> Com base nas condições climáticas de um dia nublado e úmido em São Paulo, e nas recomendações fornecidas, sugiro as seguintes atividades:
> 
> Atividade Recomendada: Explorar museus, fazer compras em shoppings ou passear em parques com áreas cobertas.
> 
> Locais para Realizar a Atividade:
> 
> Museus:
> *   **MASP (Museu de Arte de São Paulo Assis Chateaubriand)**
> *   **Pinacoteca do Estado**
> *   **Museu Afro Brasil**
> *   **Museu da Língua Portuguesa**
> *   **MIS - Museu da Imagem e do Som**
> *   **Museu Catavento**
> *   **Museu de Arte Contemporânea (MAC-USP)**
> *   **Japan House**
> *   **Museu do Futebol**
> *   **Museu de Arte Sacra de São Paulo**
> *   **Memorial da Resistência de São Paulo**
> 
> Shoppings:
> 
> *   **Shopping Iguatemi**
> *   **Shopping JK Iguatemi**
> *   **Shopping Cidade Jardim**
> *   **Shopping Pátio Higienópolis**
> *   **Shopping Cidade São Paulo**
> *   **Shopping Eldorado**
> *   **Morumbi Shopping**
> *   **Shopping Center Norte**
> *   **Shopping Interlagos**
> *   **Bourbon Shopping São Paulo**
> *   **Shopping Vila Olímpia**
> *   **Shopping Anália Franco**
> *   **Shopping Ibirapuera**
> *   **Shopping Frei Caneca**
> *   **Shopping Aricanduva**
> 
> Parques com áreas cobertas:
> 
> *   **Animália Park** (possui área coberta com brinquedos e atividades)
> *   **Parque da Aclimação** (estrutura coberta com mesas)
> * **Praça das Corujas** (possui área coberta com concreto)
> *   **Parque Villa Lobos**
> *   **Parque da Independência**
> *   **Parque do Ibirapuera**
> * **Parque Linear Aristocrata**
> * **Parque Paraisópolis**
> * **Parque Augusta-Prefeito Bruno Covas**
> * **Parque Alto da Boa Vista**
> * **Parque Linear Água Podre Ypuera**
> * **Parque Piqueri**
> * **Parque do Carmo**
> * **Parque Vila do Rodeio**
> 
> Outras opções:
> 
> *   **Mercado Municipal de São Paulo:** Para explorar a gastronomia local.
> *   **Cinesala:** Para assistir a um filme em um ambiente acolhedor.
> *   **Casa de Pedra:** Para atividades de escalada indoor.
> *   **Planetário do Carmo:** Para sessões sobre astronomia.
> 
> 


--------------------------------------------------------------------
